In [2]:
import numpy as np
import pandas as pd
import panel as pn

pn.extension('tabulator')
import hvplot.pandas

In [42]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():
    df = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')
    pn.state.cache['data'] = df.copy()
else: 
    df = pn.state.cache['data']

In [43]:
df.head()

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1949,AFG,7624058.0,NaN,NaN,NaN,0.015,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1950,AFG,7752117.0,9.421400e+09,NaN,NaN,0.084,0.070,475.0,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1951,AFG,7840151.0,9.692280e+09,NaN,NaN,0.092,0.007,8.7,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1952,AFG,7935996.0,1.001733e+10,NaN,NaN,0.092,0.000,0.0,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1953,AFG,8039684.0,1.063052e+10,NaN,NaN,0.106,0.015,16.0,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [44]:
df.shape

(26008, 60)

In [45]:
df = df.fillna(0)

In [46]:
df['gdp_per_capita'] = np.where(df.population != 0, df.gdp / df.population, 0)

In [47]:
df.head()

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share,gdp_per_capita
0,Afghanistan,1949,AFG,7624058.0,0.000000e+00,0.0,0.0,0.015,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Afghanistan,1950,AFG,7752117.0,9.421400e+09,0.0,0.0,0.084,0.070,475.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1215.332543
2,Afghanistan,1951,AFG,7840151.0,9.692280e+09,0.0,0.0,0.092,0.007,8.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1236.236369
3,Afghanistan,1952,AFG,7935996.0,1.001733e+10,0.0,0.0,0.092,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1262.264378
4,Afghanistan,1953,AFG,8039684.0,1.063052e+10,0.0,0.0,0.106,0.015,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1322.255925


## CO2 emission overtime by continent

In [48]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [49]:
#define panel widgets

year_slider = pn.widgets.IntSlider(name='Year', start=1750, end=2020, step=5, value=1850)
year_slider

IntSlider(end=2020, name='Year', start=1750, step=5, value=1850)

In [50]:
#radio btns for co2 measures

# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['co2', 'co2_per_capita',],
    button_type='success'
)
yaxis_co2

RadioButtonGroup(button_type='success', name='Y axis', options=['co2', 'co2_per_capita'], value='co2')

In [53]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [54]:
co2_pipeline

In [55]:
co2_plot = co2_pipeline.hvplot(x = 'year', by='country', y=yaxis_co2,line_width=2, title="CO2 emission by continent")
co2_plot

In [56]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
co2_table